# Using the Model

> This notebook explains how one can load a SysML v2 model and interact with the data

In [ ]:
from pathlib import Path

import pymbe.api as pm

In [ ]:
MODEL_PATH = Path(pm.__file__).parent / "../../tests/fixtures/Kerbal.json"

In [ ]:
model = pm.Model.load_from_file(MODEL_PATH)
model

all the model elements are stored under `model.elements` and can be retrieved by their SysML identifier

In [ ]:
an_element = model.elements["20511137-d751-4baf-ac4b-994a8e65ad3d"]
an_element

and their properties are resolved to the appropriate element

In [ ]:
an_element.relatedElement[0].value

In [ ]:
assert an_element.target[0].reverseReturnParameterMembership[0] == an_element.relatedElement[0]

In [ ]:
# TODO: REPR the relationships as (a)-[r:RELTYPE {name: a.name + '<->' + b.name}]->(b)